# Step 1: Load the model

In [10]:
import os
os.chdir('/workspaces/Adversraial_Research/')


from Adversarial_Observation.utils import build_cnn
import torch

model = build_cnn()
model.load_state_dict(torch.load('./examples/MNIST_CNN.pt'))
print(model)

Sequential(
  (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=784, out_features=10, bias=True)
  (8): Softmax(dim=1)
)


# Step 2: Generate a Cost Function

In [33]:
import torch
import numpy as np

def costFunc(model: torch.nn.Module, input: torch.Tensor) -> np.ndarray:
    """
    This function takes a model and a tensor input, reshapes the tensor to be a 28 x 28 image of batch size 1 and passes that through
    the model. Then, it returns the output of the column of interest (endValue) as a numpy array.
    """
    input = torch.reshape(input, (1, 1, 28, 28)).to(torch.float32)
    out = model(input)
    return out.detach().numpy()[0][2]

# Step 3: generate an attack swarm with random initializations

In [34]:
import  Adversarial_Observation.Swarm_Observer.swarm as swarm

numParticles = 10
numDimensions = 784
startingPositions = torch.rand(numParticles, numDimensions)
velocity = torch.rand(numParticles, numDimensions)

attackPSO = swarm.PSO(startingPositions, velocity, costFunc, model)

bestinitialguess = torch.argmax(model(attackPSO.pos_best_g.reshape(1,1,28,28)))
print(f'best initial guess is {bestinitialguess}')

best initial guess is 3


In [35]:
attackLoop = 1000
for i in range(attackLoop):
    attackPSO.step()


In [36]:
optimizedGuess = torch.argmax(model(attackPSO.pos_best_g.reshape(1,1,28,28)))

print(f'optimized guess is {optimizedGuess}')

optimized guess is 2


# Step 4: generate an attack swarm with a single initialization

In [37]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor

transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
mnist = MNIST('./data', train=True, download=True, transform=transform)

# get all the 4s
four = mnist.data[mnist.targets == 3]

In [38]:

numParticles = 10
numDimensions = 784

startingPositions = four[0:numParticles].reshape(numParticles, numDimensions).to(torch.float32)
velocity = torch.rand(numParticles, numDimensions)

attackPSO = swarm.PSO(startingPositions, velocity, costFunc, model)


bestinitialguess = torch.argmax(model(attackPSO.pos_best_g.reshape(1,1,28,28)))
print(f'best initial guess is {bestinitialguess}')


best initial guess is 3


In [39]:
for i in range(attackLoop):
    attackPSO.step()


In [40]:
optimizedGuess = torch.argmax(model(attackPSO.pos_best_g.reshape(1,1,28,28)))

print(f'optimized guess is {optimizedGuess}')

optimized guess is 2
